In [1]:
from scipy import stats
from torch import distributions, manual_seed, tensor

# 1 Book SCM

In [2]:
class BookSCM:
    def __init__(self, random_seed: int = None):
        self.random_seed = random_seed
        self.u_0 = distributions.Uniform(tensor([0.0]), tensor([1.0]))
        self.u_1 = distributions.Normal(tensor([0.0]), tensor([1.0]))

    def sample(self, sample_size: int = 100):
        if self.random_seed:
            manual_seed(self.random_seed)

        u_0 = self.u_0.sample((sample_size,))
        u_1 = self.u_1.sample((sample_size,))

        a = u_0 > .61
        b = (a + .5*u_1) > .2
        return a, b

In [3]:
scm = BookSCM(random_seed=45)

In [4]:
buy_book_a, buy_book_b = scm.sample(100)

In [5]:
buy_book_a.shape, buy_book_b.shape

(torch.Size([100, 1]), torch.Size([100, 1]))

In [6]:
# Let's compute probabilities
proba_book_a = buy_book_a.sum() / buy_book_a.shape[0]
proba_book_b = buy_book_b.sum() / buy_book_b.shape[0]

print(f'Probability of buying book A: {proba_book_a:0.3f}')
print(f'Probability of buying book B: {proba_book_b:0.3f}')

# ...and conditional probabilities
proba_book_a_given_book_b = (buy_book_a[buy_book_b]).sum() / buy_book_b.sum()
proba_book_b_given_book_a = (buy_book_b[buy_book_a]).sum() / buy_book_a.sum()

print(f'Probability of buying book A given B: {proba_book_a_given_book_b:0.3f}')
print(f'Probability of buying book B given A: {proba_book_b_given_book_a:0.3f}')

Probability of buying book A: 0.370
Probability of buying book B: 0.520
Probability of buying book A given B: 0.673
Probability of buying book B given A: 0.946


## 1.1 Conditioning

* Same graph, different relations

$$U_0 \sim N(0,1)$$
$$U_1 \sim N(0,1)$$
$$A \coloneqq U_0$$
$$B \coloneqq 5A + U_1$$

In [7]:
SAMPLE_SIZE = 100
manual_seed(45)

u_0 = distributions.Normal(tensor([0.0]), tensor([1.0])).sample((SAMPLE_SIZE,))
u_1 = distributions.Normal(tensor([0.0]), tensor([1.0])).sample((SAMPLE_SIZE,))

a = u_0
b = 5*a + u_1

r, p = stats.pearsonr(a, b)
print(f'Mean of B before any intervention: {b.mean():.3f}')
print(f'Variance of B before any intervention: {b.var():.3f}')
print(f'Correlation between A and B:\nr = {r[0]:.3f}; p = {p[0]:.3f}\n')

Mean of B before any intervention: -0.021
Variance of B before any intervention: 23.892
Correlation between A and B:
r = 0.977; p = 0.000



### With intervention on A

In [8]:
a = tensor([1.5]*SAMPLE_SIZE).reshape(-1,1)
b = 5*a + u_1

print(f'Mean of B after intervention on A: {b.mean():.3f}')
print(f'Variance of B after intervention on A: {b.var():.3f}')

Mean of B after intervention on A: 7.509
Variance of B after intervention on A: 1.099


### Intervening on B

In [9]:
a = u_0
b = u_1

r, p = stats.pearsonr(a, b)
print(f'Mean of B after intervention on B: {b.mean():.3f}')
print(f'Variance of B after intervention on B: {b.var():.3f}')
print(f'Correlation between A and B after intervening on B:\nr = {r[0]:.3f}; p = {p[0]:.3f}\n')

Mean of B after intervention on B: 0.009
Variance of B after intervention on B: 1.099
Correlation between A and B after intervening on B:
r = -0.006; p = 0.952



## Remarks

Intervening on A modified the mean and variance of B, but intervening on B reduced the correlation between A abd B to almost zero. This indicates that after this intervention, A and B became linearly independent. This result suggests that there is a causal link from A to B but no causal link from B to A.

# 2 Conterfactuals

In [10]:
class CounterfactualSCM:
    def abduct(self, t, y):
        return (t + y - 1)/(2*t - 1)
    def modify(self, t):
        return lambda u: t * u + (t - 1) * (u - 1)
    def predict(self, u, t):
        return self.modify(t)(u)

In [11]:

coffee = CounterfactualSCM()
t = 1
y = 1
u = coffee.abduct(t=t, y=y)
print(u)


1.0


In [12]:
coffee.predict(u=u, t=0)

0.0